In [40]:
''' 
Use Case: Stateful Database Bot for Data Insights & SQL Generation
Overview:
A stateful AI assistant integrated with the company’s data ecosystem that can answer questions and generate SQL queries based on schema knowledge, historical queries, and context. This tool empowers both technical and non-technical users to explore data independently and accurately.

Capabilities:

Understand natural language questions about data.

Maintain context across conversations (e.g., table relationships, previous questions).

Generate and explain SQL queries.

Help users explore available datasets and metadata.

Example Interactions:

User: What data do we have in our system and where is it coming from?
Bot:

Lists available datasets, e.g., orders, users, transactions.

Describes their sources: “Orders come from the e-commerce platform, updated daily via ETL at 2 AM.”

User: Give me a query that counts the number of orders per customer.
Bot:

sql
Copy
Edit
SELECT customer_id, COUNT(*) AS order_count
FROM orders
GROUP BY customer_id;
User: Now show me only customers with more than 5 orders.
Bot:

sql
Copy
Edit
SELECT customer_id, COUNT(*) AS order_count
FROM orders
GROUP BY customer_id
HAVING COUNT(*) > 5;
User: What fields are available in the transactions table?
Bot:

Lists: transaction_id, customer_id, amount, currency, timestamp, etc.

'''

In [23]:
import os
from dotenv import load_dotenv
from openai import OpenAI



In [24]:
# Load .env and get the API key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Initialize OpenAI client
client = OpenAI(api_key=api_key)


In [25]:
# Define Generic Schema (Mocked Metadata)
def load_generic_schema():
    return [
        {
            "source": "GCP_BigQuery",
            "table_name": "customers",
            "description": "Customer details including region and demographics.",
            "columns": [
                {"name": "customer_id", "type": "STRING"},
                {"name": "name", "type": "STRING"},
                {"name": "region", "type": "STRING"},
                {"name": "signup_date", "type": "DATE"},
            ]
        },
        {
            "source": "AWS_Redshift",
            "table_name": "orders",
            "description": "Order transactions placed by customers.",
            "columns": [
                {"name": "order_id", "type": "STRING"},
                {"name": "customer_id", "type": "STRING"},
                {"name": "product_id", "type": "STRING"},
                {"name": "order_date", "type": "DATE"},
                {"name": "amount", "type": "FLOAT"},
            ]
        },
        {
            "source": "AWS_RDS",
            "table_name": "products",
            "description": "Product catalog.",
            "columns": [
                {"name": "product_id", "type": "STRING"},
                {"name": "product_name", "type": "STRING"},
                {"name": "category", "type": "STRING"},
                {"name": "price", "type": "FLOAT"},
            ]
        }
    ]

#

In [26]:
# Format Schema for LLM Context
def format_schema(schema):
    formatted = ""
    for table in schema:
        formatted += f"\nTable: `{table['table_name']}` ({table['source']})\n"
        formatted += f" Description: {table['description']}\n"
        for col in table['columns']:
            formatted += f"    - {col['name']} ({col['type']})\n"
    return formatted



In [35]:

# Generate SQL Using OpenAI GPT
def generate_sql(user_input, schema_context):
    prompt = f"""
You are a helpful data assistant. Based on the schema below, write a SQL queries when needed and answer any question the user's asks regarding the data.

Schema:
{schema_context}

User Question:
{user_input}

SQL Query:
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens= 250
    )
    
    return response.choices[0].message.content.strip()



In [28]:


# Main CLI Loop
def main():
    schema = load_generic_schema()
    schema_context = format_schema(schema)

    print("Enterprise Data Assistant (NL ➜ SQL)\n")
    while True:
        user_input = input("Ask a question about your data (or 'exit'): ").strip()
        if user_input.lower() in ['exit', 'quit']:
            break
        sql = generate_sql(user_input, schema_context)
        print("\nGenerated SQL:\n", sql, "\n" + "-"*50 + "\n")


In [29]:
# Give information about all the tables available in the system?
main()

Enterprise Data Assistant (NL ➜ SQL)


Generated SQL:
 As a data assistant, I can't execute SQL queries but I can provide you with the information about the tables available in your systems based on the schema you provided.

1. `customers` table (stored in Google Cloud Platform's BigQuery): This table contains details about your customers, including their unique ID (`customer_id`), their name (`name`), the region they are from (`region`), and the date they signed up (`signup_date`).

2. `orders` table (stored in Amazon Web Services' Redshift): This table contains information about the transactions placed by customers. It includes the unique ID of the order (`order_id`), the unique ID of the customer who placed the order (`customer_id`), the unique ID of the product ordered (`product_id`), the date of the order (`order_date`), and the amount of the order (`amount`).

3. `products` table (stored in Amazon Web Services' RDS): This table contains your product catalog. It includes the uniqu

In [30]:
# Give me the number of orders for each user
main()

Enterprise Data Assistant (NL ➜ SQL)


Generated SQL:
 SELECT customer_id, COUNT(order_id) as number_of_orders
FROM orders
GROUP BY customer_id; 
--------------------------------------------------



In [31]:
# What are your functions
main()

Enterprise Data Assistant (NL ➜ SQL)


Generated SQL:
 As a data assistant, I don't have specific functions in the traditional sense. However, I can assist with a variety of tasks related to data management and analysis. Here are some examples:

1. Writing SQL queries: I can write SQL queries to retrieve, insert, update, or delete data from a database. For example, if you want to know the total sales for a specific product, I can write a query to calculate that.

2. Data analysis: I can help analyze data to find trends, patterns, or insights. This could involve statistical analysis, data visualization, or machine learning.

3. Data management: I can help manage data by organizing it, ensuring it's clean and accurate, and setting up databases or other data storage systems.

4. Data integration: I can help integrate data from different sources, such as merging data from different databases or converting data between different formats.

5. Answering questions: I can answer questions about

In [39]:
# Give me all available orders
main()

Enterprise Data Assistant (NL ➜ SQL)


Generated SQL:
 SELECT * FROM orders; 
--------------------------------------------------

